In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats
from scipy.optimize import minimize


import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

$$X= ``Número\ de\ gols\ do\ time\ de\ casa (i)" \sim Poisson(\lambda_x)$$
$$Y= ``Número\ de\ gols\ do\ time\ visitante (j)" \sim Poisson(\lambda_y)$$
$\lambda_x = \alpha_i\beta_j\gamma$\
$\lambda_y = \alpha_j\beta_i$

onde:\
$\alpha_i = Ataque\ do\ time\ i$\
$\beta_j = Defesa\ do\ time\ j$\
$\gamma = Fator\ casa\ do\ time\ i$

$$P(X=x)=\frac{(\alpha_i\beta_j\gamma)e^{-\alpha_i\beta_j\gamma}}{x!}\qquad P(Y=y)=\frac{(\alpha_j\beta_i)e^{-\alpha_j\beta_i}}{y!}\\$$

$$P(X=x, Y=y)=P(X=x)P(Y=y)=\frac{(\alpha_i\beta_j\gamma)(\alpha_j\beta_i) e^{-(\alpha_i\beta_j\gamma+\alpha_j\beta_i)}}{x!y!}$$

# Imports

In [ ]:
tabela = pd.read_excel('dados/tabela/tabela-rodada_26.xlsx')
tabela_casa = pd.read_excel('dados/tabela/tabela_casa-rodada_27.xlsx')
historico_casa = pd.read_excel('dados/historico/historicocasa-rodada_26.xlsx')
historico = pd.read_excel('dados/historico/historico-rodada_26.xlsx')
rodadas = pd.read_excel('dados/previsoes/todas_rodadas.xlsx')
rodadas_futuras = pd.read_excel('dados/previsoes/rodadas_futuras-rodada-27.xlsx')

rodada_atual = tabela['MP'].max()

# Modelo

In [ ]:
def rho_dependencia(x, y, lambda_x, lambda_y, rho):
    if x==0 and y==0:
        return 1- (lambda_x * lambda_y * rho)
    elif x==0 and y==1:
        return 1 + (lambda_x * rho)
    elif x==1 and y==0:
        return 1 + (lambda_y * rho)
    elif x==1 and y==1:
        return 1 - rho
    else:
        return 1.0

In [ ]:
def parametros_iniciais(par_x, par_y, peso_x, peso_y, fator_casa):
  dem_x = (sum(peso_x[0])+sum(peso_x[1]))
  dem_y = (sum(peso_y[0])+sum(peso_y[1]))
  times, att_i, def_i, casa_i, att_j, def_j = [], [], [], [], [], []


  for time in tabela['Squad']:
    times.append(time)
    att_i.append(float(sum(tabela.loc[tabela['Squad'] == time, par_x[0][i]]*peso_x[0][i] for i in range(len(par_x[0])))/dem_x))
    def_i.append(float(sum(tabela.loc[tabela['Squad'] == time, par_y[1][i]]*peso_y[1][i] for i in range(len(par_y[1])))/dem_y))
    casa_i.append(((1/int(tabela_casa.loc[tabela_casa['Squad'] == time, fator_casa[0]]))*0.5)+1)
    att_j.append(float(sum(tabela.loc[tabela['Squad'] == time, par_y[0][i]]*peso_y[0][i] for i in range(len(par_y[0])))/dem_y))
    def_j.append(float(sum(tabela.loc[tabela['Squad'] == time, par_x[1][i]]*peso_x[1][i] for i in range(len(par_x[1])))/dem_x))
  
  data = {
    'time': times,
    'att_i': att_i,
    'def_i': def_i,
    'casa_i': casa_i,
    'att_j': att_j,
    'def_j': def_j,
  }

  data = pd.DataFrame(data)
  return data

In [126]:
par_x = [['xG_total', 'xG_casa', 'GF_total', 'GF_casa', 'xG_5total', 'xG_5casa', 'GF_5total', 'GF_5casa'], #Ataque casa
        ['GA_total', 'GA_fora', 'xG_total', 'xG_fora', 'xG_5total', 'xG_5fora', 'GA_5total', 'GA_5fora']]  #Defesa fora
peso_x = [[1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0]]


par_y = [['xG_total', 'xG_fora', 'GF_total', 'GF_fora', 'xG_5total', 'xG_5fora', 'GF_5total', 'GF_5fora'], #Ataque fora
        ['GA_total', 'GA_casa', 'xG_total', 'xG_casa', 'xG_5total', 'xG_5casa', 'GA_5total', 'GA_5casa']]  #Defesa casa
peso_y = [[1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0]]

fator_casa = ['Rk_casa']

par_iniciais = parametros_iniciais(par_x, par_y, peso_x, peso_y, fator_casa)


parametros = np.concatenate((par_iniciais['att_i'],  #Alpha 
                             par_iniciais['att_j'],  #Alpha 
                             par_iniciais['def_i'],  #Beta 
                             par_iniciais['def_j'],  #Beta 
                             par_iniciais['casa_i'], #Casa  
                             np.array([0])))         #Rho

In [130]:
stats.poisson.pmf(2, )

,time,att_i,def_i,casa_i,att_j,def_j
0,Botafogo (RJ),1.226923,0.765385,1.500000,0.985577,0.697115
6,Fluminense,1.008036,0.887500,1.250000,0.982143,1.062500
2,Gremio,1.172115,0.950962,1.166667,1.028846,1.048077
7,Ath Paranaense,1.151637,0.993899,1.125000,0.977381,0.929762
9,Sao Paulo,1.042113,0.926042,1.100000,0.860119,0.895833
5,Fortaleza,1.122768,0.975446,1.083333,0.941071,0.914286
1,Bragantino,1.135577,0.885577,1.071429,0.993269,0.868269
3,Palmeiras,1.204464,0.924702,1.062500,1.077679,0.830655
4,Flamengo,1.086905,0.929167,1.055556,1.036310,0.970833
13,Corinthians,0.892308,0.844231,1.050000,0.725962,0.831731
